In [ ]:
#Import required librariesimport pandas as pd
import librosa

import numpy as np

from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split


import tensorflow as tf

import random as python_random

#Libraries for implementation of CNN.
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten

#Libraries for training the model
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime

import csv

import random

In [56]:
#Variable parameters
v_dataset_size = 4500 #Number of data used for tuning the model
VALIDATION_DATASET_SIZE = 500

COMBINATION_COUNT_FOR_EACH_MFCC = 30

RANDOM_SEED_VALUE = 10

NUM_OF_CLASSES = 2

In [59]:
#Global Parameters
SAMPLE_RATE = 44100

DATA_FOLDER = 'data232\\'
METADATA_FOLDER = 'metadata232\\'
METADATA_FILENAME = 'metadata232.csv'
TESTDATA_FOLDER = 'testdata230\\'
MODEL_FOLDER = 'tuningmode300\\'
SYNTH_DATA_FOLDER = 'testdatasynth00\\'
TUNING_DATA_FILEPATH = 'tuningdata.csv'

CSV_READ_CHUNK_SIZE = 2000

SAMPLE_LENGTH_SEC = 2

In [60]:
#Variable feature extraction parameters
v_n_mfcc_vals = [10,32,64,128]

#Variable hyperparameters
h_epochs_vals = [1,5,10]
h_batch_size_vals = [16,32,64]

#CNN hyperparameters
#Convolutional layers
h_layers_count_vals = [3,4,5,6]
h_filter_size_vals = [32,64]
h_kernel_size_vals = [(3,3)]
h_strides_vals = [(1,1)]
h_activation_function_vals = ['relu']
h_max_pooling_pool_size_vals = [(2,2)]
h_max_pooling_strides_vals = [(2,2)]
h_dropout_rate_vals = [0.2,0.3,0.5,0.6]

#Flattening layer
h_flatten_dropout_rate_vals = [0.2,0.4,0.5,0.6]

#Output function
h_output_activation_function_vals = ['softmax','sigmoid']

#Model training parameters
h_loss_function_vals = ['categorical_crossentropy','binary_crossentropy']
h_optimizer_vals = ['adam']

In [61]:
#Function to get MFCC data from a given single audio file
def features_extractor(audio_file_path,n_mfcc):
    #Selection of res_type => https://librosa.org/doc/main/generated/librosa.resample.html#librosa.resample
    #Faster method is selected
    audio, sample_rate = librosa.load(audio_file_path, res_type='kaiser_fast',sr=SAMPLE_RATE)
    
    if(len(audio)>sample_rate*SAMPLE_LENGTH_SEC):
        audio = audio[:sample_rate*SAMPLE_LENGTH_SEC]
    
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=n_mfcc)
    return mfccs_features

In [62]:
#Script to automate measuring modle accuracy for synthesizer data.
import os
import glob

def get_synth_accuracy(data_folder,model,n_mfcc,input_width):
    X_synthtest = []
    y_synthtest = []
    
    #Picking only .wav files
    wav_file_paths = glob.glob(data_folder+'*.wav')
    
    for wav_file_path in wav_file_paths:
        wav_file_features1 = features_extractor(wav_file_path,n_mfcc)    
        wav_file_features = pad_features(wav_file_features1,n_mfcc,input_width)
    
        X_synthtest.append(wav_file_features)
    
        wav_file_name = os.path.basename(wav_file_path)
        split_file_name = wav_file_name.split('_')

        #For easier identification;
        #-a non-chorus file name is in the format 'v_*.wav'
        #-a chorus file name is in the format 'c_*.wav'
        #The naming convention is used to label the files here.
        if(split_file_name[0]=='v'):
            wav_file_class = 0
        elif(split_file_name[0]=='c'):
            wav_file_class = 1
        else:
            wav_file_class = -1

        #Printing prediction for each file
        label = model.predict(np.array([wav_file_features]))
        classes_x=np.argmax(label,axis=1)
        prediction_class = labelencoder.inverse_transform(classes_x)
        print(str(wav_file_name)+' || label = '+str(label)+' || prediction = '+str(prediction_class))

        y_synthtest.append(wav_file_class)
        
    len_Xsynthtest = len(X_synthtest)
    len_ysynthtest = len(y_synthtest)

    X_synthtest=np.array(X_synthtest).reshape(len_Xsynthtest,n_mfcc,input_width,1)
    y_synthtest=to_categorical(labelencoder.fit_transform(y_synthtest))
    y_synthtest=y_synthtest.reshape(len_ysynthtest,2)

    synth_accuracy=model.evaluate(X_synthtest,y_synthtest,verbose=0)

    return synth_accuracy[1]

In [63]:
import math

#Idea for cleanly writing padding function is taken from the below 2 posts.
#https://stackoverflow.com/questions/59241216/padding-numpy-arrays-to-a-specific-size
#https://towardsdatascience.com/cnns-for-audio-classification-6244954665ab
def pad_features(feature_array,expected_height,expected_width):
    array_height = feature_array.shape[0]
    array_width = feature_array.shape[1]
    
    pad_height = max(expected_height-array_height,0)
    pad_height1 = math.floor(pad_height/2)
    pad_height2 = max(pad_height-pad_height1,0)
    
    pad_width = max(expected_width-array_width,0)
    pad_width1 = math.floor(pad_width/2)
    pad_width2 = max(pad_width-pad_width1,0)
    
    #print('h->'+str(pad_height)+'|| w->'+str(pad_width))
    return np.pad(array=feature_array,pad_width=((pad_height1,pad_height2),
                                                 (pad_width1,pad_width2)),mode='constant')

In [64]:
#Read stored metadata and get a Pandas dataframe
file_path = METADATA_FOLDER+METADATA_FILENAME

used_columns = ['filename','chorus']

dtypes = {
    'filename': 'str',
    'chorus': 'int'
}

data_chunks = pd.read_csv(file_path, usecols=used_columns,
                          dtype=dtypes, chunksize=CSV_READ_CHUNK_SIZE)

# concatenate the chunks into a single DataFrame
df = pd.concat(data_chunks, ignore_index=True)

In [65]:
tuning_data_header = [
    'record_no',
    'seed_val',
    'random_records_count',
    'v_n_mfcc',
    'h_layers_count',
    'h_filter_size',
    'h_kernel_size',
    'h_strides',
    'h_activation_function',
    'h_max_pooling_pool_size',
    'h_max_pooling_strides',
    'h_dropout_rate',
    'h_flatten_dropout_rate',
    'h_output_activation_function',
    'h_loss_function',
    'h_optimizer',
    'h_epochs',
    'h_batch_size',
    'm_training_duration',
    'm_validation_accuracy',
    'm_test_accuracy'
    'm_synth_accuracy'
]

#Writing header to csv file
with open(TUNING_DATA_FILEPATH, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(tuning_data_header)
    
    record_no = 1

    #Picking n_mfcc feature extraction parameter values
    for v_n_mfcc in v_n_mfcc_vals:
        #Initialization of extracted data
        extracted_X = []
        extracted_y = []        

        #Read metadata on the dataset to fetch .wav file names.
        for index, row in df.iterrows():
        #Limit fetching the data when it reaches the specified dataset size
            if (index>v_dataset_size-1):
                break

            file_name = row['filename']
            class_label = row['chorus']

            features = features_extractor(DATA_FOLDER+file_name,v_n_mfcc)    
            extracted_X.append(features)
            extracted_y.append(class_label)

        #Convert to numpy arrays
        extracted_X = np.array(extracted_X)
        extracted_y = np.array(extracted_y)    

        #Output classes
        labelencoder=LabelEncoder()
        extracted_y=to_categorical(labelencoder.fit_transform(extracted_y))

        print(extracted_X.shape)
        print(extracted_y.shape)

        X_train,X_test,y_train,y_test=train_test_split(extracted_X[:v_dataset_size-VALIDATION_DATASET_SIZE],
                                                       extracted_y[:v_dataset_size-VALIDATION_DATASET_SIZE],
                                                       test_size=0.2,random_state=1)

        #Setting seed values to get reproducible outputs and same random values for each hyperparameter set.
        np.random.seed(RANDOM_SEED_VALUE)
        tf.random.set_seed(RANDOM_SEED_VALUE)    
        python_random.seed(RANDOM_SEED_VALUE)

        #input shape of extracted mfcc 2D matrix
        input_shape = (v_n_mfcc,extracted_X.shape[2],1)
                
        seed_val = 0
        random_records_count = 0
        
        #Picking random hyperparameters
        while (random_records_count<COMBINATION_COUNT_FOR_EACH_MFCC):
            random.seed(seed_val)
            
            h_layers_count = random.choice(h_layers_count_vals)
            h_filter_size = random.choice(h_filter_size_vals)
            h_kernel_size = random.choice(h_kernel_size_vals)
            h_strides = random.choice(h_strides_vals)
            h_activation_function = random.choice(h_activation_function_vals)
            h_max_pooling_pool_size = random.choice(h_max_pooling_pool_size_vals)
            h_max_pooling_strides = random.choice(h_max_pooling_strides_vals)
            h_dropout_rate = random.choice(h_dropout_rate_vals)
            h_flatten_dropout_rate = random.choice(h_flatten_dropout_rate_vals)
            h_output_activation_function = random.choice(h_output_activation_function_vals)
            h_loss_function = random.choice(h_loss_function_vals)
            h_optimizer = random.choice(h_optimizer_vals)
            h_epochs = random.choice(h_epochs_vals)
            h_batch_size = random.choice(h_batch_size_vals)
            
            #Setting seed values to get reproducible outputs and same random values for each hyperparameter set.
            np.random.seed(RANDOM_SEED_VALUE)
            tf.random.set_seed(RANDOM_SEED_VALUE)    
            python_random.seed(RANDOM_SEED_VALUE)
            
            print('PROCESSING RECORD=> '+str(record_no)+'---------------------------------n_mfcc = '+str(v_n_mfcc))
            
            #DEBUG    
            print('random_records_count=> '+str(random_records_count))
            print('h_layers_count=> '+str(h_layers_count))
            print('h_filter_size=> '+str(h_filter_size))
            print('h_kernel_size=> '+str(h_kernel_size))
            print('h_strides=> '+str(h_strides))
            print('h_activation_function=> '+str(h_activation_function))
            print('h_max_pooling_pool_size=> '+str(h_max_pooling_pool_size))
            print('h_max_pooling_strides=> '+str(h_max_pooling_strides))
            print('h_dropout_rate=> '+str(h_dropout_rate))
            print('h_flatten_dropout_rate=> '+str(h_flatten_dropout_rate))
            print('h_output_activation_function=> '+str(h_output_activation_function))
            print('h_loss_function=> '+str(h_loss_function))
            print('h_optimizer=> '+str(h_optimizer))
            print('h_epochs=> '+str(h_epochs))
            print('h_batch_size=> '+str(h_batch_size))
            
            try:
                #Reset the model
                model=None
                
                # create the model
                model = Sequential()

                #CNN Layer 1 with mirrored padding after the input layer
                conv_layer1 = Conv2D(filters=h_filter_size,
                                     kernel_size=h_kernel_size,
                                     strides=h_strides,
                                     padding='same',
                                     data_format='channels_last',
                                     activation=h_activation_function,
                                     input_shape=input_shape)
                maxpool_layer1 = MaxPooling2D(pool_size=h_max_pooling_pool_size,
                                              strides=h_max_pooling_strides, padding='valid')
                dropout_layer1 = Dropout(rate=h_dropout_rate)

                model.add(conv_layer1)

                #Add convolutional layers
                for layer_no in range(1,h_layers_count):
                    conv_layer_i = Conv2D(filters=h_filter_size*(layer_no+1),
                                          kernel_size=h_kernel_size,
                                          strides=h_strides,
                                          padding='same',
                                          data_format='channels_last',
                                          activation=h_activation_function)
                    print('   Adding conv layers--------')
                    print('   layer_no =>'+str(layer_no))

                    maxpool_layer_i = MaxPooling2D(pool_size=h_max_pooling_pool_size, strides=h_max_pooling_strides,
                                                   padding='valid')
                    dropout_layer_i = Dropout(rate=h_dropout_rate)

                    model.add(conv_layer_i)
                    model.add(maxpool_layer_i)
                    model.add(dropout_layer_i)

                # Add a flattening layer after the dropout layer
                flatten_layer = Flatten()
                dropout_layer = Dropout(rate=h_flatten_dropout_rate)

                #Output Layer
                output_layer=Dense(units=NUM_OF_CLASSES,activation=h_output_activation_function)

                model.add(flatten_layer)
                model.add(dropout_layer)
                model.add(output_layer)

                #Compiling the model
                model.compile(loss=h_loss_function,metrics=['accuracy'],optimizer=h_optimizer)
                model.summary()

                checkpointer = ModelCheckpoint(filepath=MODEL_FOLDER+str(record_no)+'_model'+'.hdf5',
                                               verbose=1,
                                               save_best_only=True)
                start = datetime.now()

                #Training the model
                model.fit(X_train,
                          y_train,
                          batch_size=h_batch_size,
                          epochs=h_epochs,
                          validation_data=(X_test, y_test),
                          callbacks=[checkpointer],
                          verbose=1)


                m_training_duration = datetime.now() - start
                print('Training for record '+str(record_no)+' completed in time: ', m_training_duration)

                #Measuring the performance of the trained model

                #Validation accuracy
                validation_accuracy = model.evaluate(X_test,y_test,verbose=0)
                m_validation_accuracy = validation_accuracy[1]

                #Test accuracy
                test_accuracy = model.evaluate(extracted_X[v_dataset_size-VALIDATION_DATASET_SIZE:],
                                              extracted_y[v_dataset_size-VALIDATION_DATASET_SIZE:],
                                              verbose=0)
                m_test_accuracy = test_accuracy[1]

                m_synth_accuracy = get_synth_accuracy(SYNTH_DATA_FOLDER,model,v_n_mfcc,extracted_X.shape[2])

                tuning_data_row = [record_no,
                                   seed_val,
                                   random_records_count,
                                      v_n_mfcc,
                                      h_layers_count,
                                      h_filter_size,
                                      h_kernel_size,
                                      h_strides,
                                      h_activation_function,
                                      h_max_pooling_pool_size,
                                      h_max_pooling_strides,
                                      h_dropout_rate,
                                      h_flatten_dropout_rate,
                                      h_output_activation_function,
                                      h_loss_function,
                                      h_optimizer,
                                      h_epochs,
                                      h_batch_size,
                                      m_training_duration,
                                      m_validation_accuracy,
                                      m_test_accuracy,
                                      m_synth_accuracy]

                #Writing data row to csv file
                writer.writerow(tuning_data_row)

                

                record_no +=1
                seed_val += 1
                random_records_count += 1
                
                
                print('COMPLETED SUCCESSFULLY----------------------------.')
            except Exception as e:
                model=None
                seed_val += 1
                print(e)
                print('EXCEPTION OCCURRED--------------------------------.')
                #If any exception occurrs, the selected parameter combination is skipped and the next one is selected.
                continue
                
                

(4500, 10, 173)
(4500, 2)
PROCESSING RECORD=> 1---------------------------------n_mfcc = 10
random_records_count=> 0
h_layers_count=> 6
h_filter_size=> 64
h_kernel_size=> (3, 3)
h_strides=> (1, 1)
h_activation_function=> relu
h_max_pooling_pool_size=> (2, 2)
h_max_pooling_strides=> (2, 2)
h_dropout_rate=> 0.6
h_flatten_dropout_rate=> 0.5
h_output_activation_function=> softmax
h_loss_function=> categorical_crossentropy
h_optimizer=> adam
h_epochs=> 1
h_batch_size=> 16
   Adding conv layers--------
   layer_no =>1
   Adding conv layers--------
   layer_no =>2
   Adding conv layers--------
   layer_no =>3
   Adding conv layers--------
   layer_no =>4
Exception encountered when calling layer "max_pooling2d_134" (type MaxPooling2D).

Negative dimension size caused by subtracting 2 from 1 for '{{node max_pooling2d_134/MaxPool}} = MaxPool[T=DT_FLOAT, data_format="NHWC", explicit_paddings=[], ksize=[1, 2, 2, 1], padding="VALID", strides=[1, 2, 2, 1]](Placeholder)' with input shapes: [?,1,21,32

1/1 [==============================] - 0s 19ms/step
v_15.wav || label = [[0.99552155 0.00447851]] || prediction = [0]
1/1 [==============================] - 0s 21ms/step
v_2.wav || label = [[0.37078595 0.6292141 ]] || prediction = [1]
1/1 [==============================] - 0s 21ms/step
v_3.wav || label = [[0.6423629 0.3576371]] || prediction = [0]
1/1 [==============================] - 0s 17ms/step
v_4.wav || label = [[0.11105352 0.8889464 ]] || prediction = [1]
1/1 [==============================] - 0s 18ms/step
v_5.wav || label = [[0.1107431  0.88925683]] || prediction = [1]
1/1 [==============================] - 0s 22ms/step
v_6.wav || label = [[0.29477176 0.7052282 ]] || prediction = [1]
1/1 [==============================] - 0s 20ms/step
v_7.wav || label = [[0.27522963 0.72477037]] || prediction = [1]
1/1 [==============================] - 0s 18ms/step
v_8.wav || label = [[0.9206358  0.07936422]] || prediction = [0]
1/1 [==============================] - 0s 19ms/step
v_9.wav || la

1/1 [==============================] - 0s 27ms/step
c_9.wav || label = [[0.2769575  0.72304255]] || prediction = [1]
1/1 [==============================] - 0s 17ms/step
v_1.wav || label = [[0.23010974 0.76989025]] || prediction = [1]
1/1 [==============================] - 0s 16ms/step
v_10.wav || label = [[9.9973863e-01 2.6134125e-04]] || prediction = [0]
1/1 [==============================] - 0s 29ms/step
v_11.wav || label = [[0.3153596  0.68464035]] || prediction = [1]
1/1 [==============================] - 0s 31ms/step
v_12.wav || label = [[0.30700904 0.6929909 ]] || prediction = [1]
1/1 [==============================] - 0s 29ms/step
v_13.wav || label = [[0.7673616  0.23263845]] || prediction = [0]
1/1 [==============================] - 0s 15ms/step
v_14.wav || label = [[0.34210178 0.65789825]] || prediction = [1]
1/1 [==============================] - 0s 16ms/step
v_15.wav || label = [[9.9985170e-01 1.4823469e-04]] || prediction = [0]
1/1 [==============================] - 0s 16ms

   Adding conv layers--------
   layer_no =>3
   Adding conv layers--------
   layer_no =>4
Exception encountered when calling layer "max_pooling2d_162" (type MaxPooling2D).

Negative dimension size caused by subtracting 2 from 1 for '{{node max_pooling2d_162/MaxPool}} = MaxPool[T=DT_FLOAT, data_format="NHWC", explicit_paddings=[], ksize=[1, 2, 2, 1], padding="VALID", strides=[1, 2, 2, 1]](Placeholder)' with input shapes: [?,1,21,160].

Call arguments received by layer "max_pooling2d_162" (type MaxPooling2D):
  • inputs=tf.Tensor(shape=(None, 1, 21, 160), dtype=float32)
EXCEPTION OCCURRED--------------------------------.
PROCESSING RECORD=> 6---------------------------------n_mfcc = 10
random_records_count=> 5
h_layers_count=> 4
h_filter_size=> 64
h_kernel_size=> (3, 3)
h_strides=> (1, 1)
h_activation_function=> relu
h_max_pooling_pool_size=> (2, 2)
h_max_pooling_strides=> (2, 2)
h_dropout_rate=> 0.3
h_flatten_dropout_rate=> 0.4
h_output_activation_function=> softmax
h_loss_function=> 

1/1 [==============================] - 0s 20ms/step
v_6.wav || label = [[5.486926e-18 1.000000e+00]] || prediction = [1]
1/1 [==============================] - 0s 19ms/step
v_7.wav || label = [[1.4959407e-16 1.0000000e+00]] || prediction = [1]
1/1 [==============================] - 0s 18ms/step
v_8.wav || label = [[9.9996388e-01 3.6071586e-05]] || prediction = [0]
1/1 [==============================] - 0s 28ms/step
v_9.wav || label = [[2.6043426e-22 1.0000000e+00]] || prediction = [1]
COMPLETED SUCCESSFULLY----------------------------.
PROCESSING RECORD=> 7---------------------------------n_mfcc = 10
random_records_count=> 6
h_layers_count=> 6
h_filter_size=> 64
h_kernel_size=> (3, 3)
h_strides=> (1, 1)
h_activation_function=> relu
h_max_pooling_pool_size=> (2, 2)
h_max_pooling_strides=> (2, 2)
h_dropout_rate=> 0.6
h_flatten_dropout_rate=> 0.2
h_output_activation_function=> sigmoid
h_loss_function=> categorical_crossentropy
h_optimizer=> adam
h_epochs=> 1
h_batch_size=> 64
   Adding co

...

50/50 [==============================] - 169s 3s/step - loss: 0.0016 - accuracy: 0.9994 - val_loss: 0.0013 - val_accuracy: 1.0000
Epoch 5/10
50/50 [==============================] - ETA: 0s - loss: 0.0015 - accuracy: 0.9994
Epoch 5: val_loss did not improve from 0.00128
50/50 [==============================] - 179s 4s/step - loss: 0.0015 - accuracy: 0.9994 - val_loss: 0.0031 - val_accuracy: 1.0000
Epoch 6/10
50/50 [==============================] - ETA: 0s - loss: 1.1551e-04 - accuracy: 1.0000
Epoch 6: val_loss improved from 0.00128 to 0.00093, saving model to tuningmode300\115_model.hdf5
50/50 [==============================] - 167s 3s/step - loss: 1.1551e-04 - accuracy: 1.0000 - val_loss: 9.2532e-04 - val_accuracy: 1.0000
Epoch 7/10
50/50 [==============================] - ETA: 0s - loss: 4.3541e-05 - accuracy: 1.0000
Epoch 7: val_loss improved from 0.00093 to 0.00049, saving model to tuningmode300\115_model.hdf5
50/50 [==============================] - 162s 3s/step - loss: 4.3541e-0

 max_pooling2d_749 (MaxPooli  (None, 8, 10, 160)       0         
 ng2D)                                                           
                                                                 
 dropout_896 (Dropout)       (None, 8, 10, 160)        0         
                                                                 
 conv2d_750 (Conv2D)         (None, 8, 10, 192)        276672    
                                                                 
 max_pooling2d_750 (MaxPooli  (None, 4, 5, 192)        0         
 ng2D)                                                           
                                                                 
 dropout_897 (Dropout)       (None, 4, 5, 192)         0         
                                                                 
 flatten_147 (Flatten)       (None, 3840)              0         
                                                                 
 dropout_898 (Dropout)       (None, 3840)              0         
          

   Adding conv layers--------
   layer_no =>3
Model: "sequential_176"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_751 (Conv2D)         (None, 128, 173, 32)      320       
                                                                 
 conv2d_752 (Conv2D)         (None, 128, 173, 64)      18496     
                                                                 
 max_pooling2d_752 (MaxPooli  (None, 64, 86, 64)       0         
 ng2D)                                                           
                                                                 
 dropout_900 (Dropout)       (None, 64, 86, 64)        0         
                                                                 
 conv2d_753 (Conv2D)         (None, 64, 86, 96)        55392     
                                                                 
 max_pooling2d_753 (MaxPooli  (None, 32, 43, 96)       0         
 ng2D)

1/1 [==============================] - 0s 59ms/step
v_15.wav || label = [[9.9999738e-01 2.6251387e-06]] || prediction = [0]
1/1 [==============================] - 0s 58ms/step
v_2.wav || label = [[1.0000000e+00 2.6702053e-14]] || prediction = [0]
1/1 [==============================] - 0s 60ms/step
v_3.wav || label = [[1.0000000e+00 3.0710552e-14]] || prediction = [0]
1/1 [==============================] - 0s 59ms/step
v_4.wav || label = [[1.000000e+00 6.830184e-09]] || prediction = [0]
1/1 [==============================] - 0s 58ms/step
v_5.wav || label = [[1.0000000e+00 4.6947396e-10]] || prediction = [0]
1/1 [==============================] - 0s 63ms/step
v_6.wav || label = [[9.999205e-01 7.953250e-05]] || prediction = [0]
1/1 [==============================] - 0s 62ms/step
v_7.wav || label = [[0.998803   0.00119701]] || prediction = [0]
1/1 [==============================] - 0s 67ms/step
v_8.wav || label = [[1.0000000e+00 2.7036074e-15]] || prediction = [0]
1/1 [====================

 max_pooling2d_765 (MaxPooli  (None, 64, 86, 128)      0         
 ng2D)                                                           
                                                                 
 dropout_916 (Dropout)       (None, 64, 86, 128)       0         
                                                                 
 conv2d_766 (Conv2D)         (None, 64, 86, 192)       221376    
                                                                 
 max_pooling2d_766 (MaxPooli  (None, 32, 43, 192)      0         
 ng2D)                                                           
                                                                 
 dropout_917 (Dropout)       (None, 32, 43, 192)       0         
                                                                 
 flatten_151 (Flatten)       (None, 264192)            0         
                                                                 
 dropout_918 (Dropout)       (None, 264192)            0         
          